In [18]:

import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss
from omop_alchemy.model.clinical import Person, Condition_Occurrence, Measurement, Observation
from omop_alchemy.conventions.vocab_lookups import gender_lookup, ethnicity_lookup, race_lookup


In [19]:
try:
    Person(year_of_birth=10000)
except:
    print('expected fail: yob')

expected fail: yob


In [20]:
try:
    Person(month_of_birth=13)
except:
    print('expected fail: mob')

expected fail: mob


In [21]:
try:
    p = Person(gender_concept_id=100)
except:
    print('expected fail: gender')

expected fail: gender


In [22]:
'male' in gender_lookup, 8507 in gender_lookup, 'sldkfjsd' in gender_lookup

(True, True, False)

In [23]:
try:
    p = Person(race_concept_id=100)
except:
    print('expected fail: race')

expected fail: race


In [8]:
p = Person(gender_concept_id = 8532, year_of_birth=1960, month_of_birth=10, day_of_birth=3)

In [9]:
p.dob

datetime.datetime(1960, 10, 3, 0, 0)

In [10]:
p.age

{'age_total': 23140, 'age_years': 63, 'age_days': 145}

In [11]:
p2 = Person()
p2.age

{}

In [12]:
# if you don't assign the ID explicitly, you have to commit it to the DB before it will be incremented
p2

Person: person_id = None

In [17]:
from omop_alchemy import oa_config

with so.Session(oa_config.engine) as session:
    session.add(p2)
    session.commit()
    p2.person_id # TODO: this ID should be available once committed?? what's up with that?

In [32]:
# get all hemonc components from athena
import omop_alchemy as oa



ModuleNotFoundError: No module named 'pandas'

In [ ]:

def get_cols(result_obj):
    return [c for c in dir(result_obj) if c[0] != '_' and c != 'metadata']
    
def get_obj_results(cols, result_obj, field_name=''):
    results = {f'{field_name}.{c}'.strip('.'):getattr(result_obj, c) for c in cols}
    return results

def get_rec_results(result_obj, field_name=''):
    results = {}
    if hasattr(result_obj, '_real_fields'):
        for f in result_obj._real_fields:
            results.update(get_rec_results(getattr(result_obj, f), f'{field_name}.{f}'))
    elif hasattr(result_obj, '__tablename__'):
        results = get_obj_results(get_cols(result_obj), result_obj, field_name)
    else:
        results = {field_name.strip('.'): result_obj}
    return results

# TODO SQLAlchemy 1.4 intruduces a breaking change for this - returns row objects instead of query results - to update, because there are other nice things with sqlalchemy 1.4 that would make it worthwhile

def list_from_query(query_results):
    return [x[0] for x in query_results]

def dataframe_from_query(query_results):
    l = [get_rec_results(q) for q in query_results]
    return pd.DataFrame(l)


In [ ]:
oa.oa_config.log_path